In [47]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis = 1)

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
label_encoded_gender = LabelEncoder()

In [8]:
data['Gender'] = label_encoded_gender.fit_transform(data['Gender'])

In [9]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
onehot_encoded_df = OneHotEncoder()

In [25]:
geo_encoded = onehot_encoded_df.fit_transform(data[['Geography']]).toarray()

In [26]:
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [27]:
columns = onehot_encoded_df.get_feature_names_out(['Geography'])

In [29]:
geo_encoded_df = pd.DataFrame(geo_encoded , columns=columns)

In [31]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [33]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1) 

In [34]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [36]:
x = data.drop(['EstimatedSalary'],axis = 1)

In [37]:
y = data['EstimatedSalary']

In [41]:
x_train,x_test,y_train , y_test = train_test_split(x,y,test_size=0.2 , random_state=42)

In [42]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((8000, 12), (8000,), (2000, 12), (2000,))

In [43]:
scaler = StandardScaler()

In [46]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [49]:
with open('label_encoded_gender.pkl','wb') as file:
    pickle.dump(label_encoded_gender,file)

with open('onehot_encoded_geo.pkl','wb') as file:
    pickle.dump(onehot_encoded_df,file)

with open('Scaler.pkl','wb') as file:
    pickle.dump(scaler , file)

### ANN Regression Implementation


In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import pickle
import numpy as np

In [52]:
model = Sequential([
        Dense(64 , activation='relu',input_shape = (x_train.shape[1],)),
        Dense(32 , activation = 'relu'),
        Dense(1) 
])

In [53]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.compile(optimizer='adam',loss = 'mean_absolute_error',metrics=['mae'])

In [56]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir = 'regressionlogs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir , histogram_freq=1)

In [57]:
earlystopping_callback = EarlyStopping(monitor='val_loss',patience = 10 , restore_best_weights=True)

In [58]:
history = model.fit(x_train,y_train,
                    validation_data=[x_test,y_test],
                    epochs=100,
                    callbacks=[earlystopping_callback,tensorboard_callback]
                   )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 100371.5859 - mae: 100371.5859 - val_loss: 98500.1484 - val_mae: 98500.1484
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 99586.1094 - mae: 99586.1094 - val_loss: 96927.8594 - val_mae: 96927.8594
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 96813.1797 - mae: 96813.1797 - val_loss: 92832.6328 - val_mae: 92832.6328
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 91267.3438 - mae: 91267.3438 - val_loss: 85966.3516 - val_mae: 85966.3516
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 83232.4531 - mae: 83232.4531 - val_loss: 77237.3359 - val_mae: 77237.3359
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 73962.7188 - mae: 73962.7188 - val_loss: 68284.7578 - val_mae: 68284.7578
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 65178.5547 - mae: 65178.5547 - val_loss: 60621.5273 - val_mae: 60621.5273
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

In [59]:
%load_ext tensorboard
%tensorboard --logdir regressionlogs/fit/

In [60]:
test_loss,test_mae = model.evaluate(x_test,y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50336.8516 - mae: 50336.8516 


In [61]:
test_mae

50336.8515625

In [62]:
test_loss

50336.8515625

In [64]:
model.save('regression_model.h5')